## Helper script
This is a helper sript defined to find the DOY of each plume flyby to match the Cassini database documentation.

In [1]:
import pandas as pd
import re
from pathlib import Path
import pandas as pd

In [2]:
# Get the DOY of all flybys to match the CDA databases to download

flybys = [
    ("E3",  "2008-03-12 19:06:45"),
    ("E4",  "2008-08-11 21:06:30"),
    ("E5",  "2008-10-09 19:07:00"),
    ("E6",  "2008-10-31 17:15:30"),
    ("E7",  "2009-11-02 07:42:00"),
    ("E9",  "2010-04-28 19:09:30"),
    ("E12", "2010-11-30 12:06:00"),
    ("E13", "2010-12-21 13:11:00"),
    ("E14", "2011-10-01 18:40:00"),
    ("E17", "2012-03-27 18:30:00"),
    ("E18", "2012-04-14 18:08:00"),
    ("E19", "2012-05-02 18:12:00"),
    ("E21", "2015-10-28 15:22:00"),
]


for name, date_str in flybys:
    dt = pd.to_datetime(date_str)
    doy = dt.dayofyear
    print(f"{name}: Year: {dt.year} | DOY: {doy}")

E3: Year: 2008 | DOY: 72
E4: Year: 2008 | DOY: 224
E5: Year: 2008 | DOY: 283
E6: Year: 2008 | DOY: 305
E7: Year: 2009 | DOY: 306
E9: Year: 2010 | DOY: 118
E12: Year: 2010 | DOY: 334
E13: Year: 2010 | DOY: 355
E14: Year: 2011 | DOY: 274
E17: Year: 2012 | DOY: 87
E18: Year: 2012 | DOY: 105
E19: Year: 2012 | DOY: 123
E21: Year: 2015 | DOY: 301


In [3]:
# check if we downloaded the correct databases

flybys = [(n, pd.Timestamp(t)) for n, t in flybys]

root = Path("../Dataset")
vol_pattern = re.compile(r"COCDA_\d{4}")
volumes = [d for d in root.iterdir() if d.is_dir() and vol_pattern.fullmatch(d.name)]

for vol in sorted(volumes):
    event_files = sorted(vol.glob("DATA/**/CDAEVENTS_*.TAB"))

    min_t, max_t = None, None
    for f in event_files:
        try:
            # Read the time
            df = pd.read_fwf(f, colspecs=[(11, 28)], names=["EVENT_TIME"], dtype=str)
            df["EVENT_TIME"] = pd.to_datetime(df["EVENT_TIME"], errors="coerce", format="%Y-%jT%H:%M:%S")
            df = df.dropna(subset=["EVENT_TIME"])

            # determine min and maxima
            local_min = df["EVENT_TIME"].min()
            local_max = df["EVENT_TIME"].max()
            if not min_t or local_min < min_t:
                min_t = local_min
            if not max_t or local_max > max_t:
                max_t = local_max

        except Exception as e:
            print(f"[ERROR] Failed parsing {f.name}: {e}")

    if not min_t or not max_t:
        print(f"[INFO] {vol.name}: Could not determine time range")
        continue

    print(f"\n{vol.name}: {min_t:%Y-%m-%d %H:%M:%S} -> {max_t:%Y-%m-%d %H:%M:%S}")

    # Check which flybys are in this range
    inside = [f"{n} ({t:%Y-%m-%d %H:%M:%S})" for n, t in flybys if min_t <= t <= max_t]
    if inside:
        print(f"  Flybys inside {vol.name}: {', '.join(inside)}")
    else:
        print(f"  No flybys inside {vol.name}.")


COCDA_0007: 2005-01-01 00:02:42 -> 2005-03-31 23:59:49
  No flybys inside COCDA_0007.

COCDA_0012: 2005-07-01 00:00:44 -> 2005-07-21 23:58:48
  No flybys inside COCDA_0012.

COCDA_0045: 2008-03-02 01:01:41 -> 2008-03-31 23:57:25
  Flybys inside COCDA_0045: E3 (2008-03-12 19:06:45)

COCDA_0050: 2008-08-04 13:36:30 -> 2008-08-26 13:39:19
  Flybys inside COCDA_0050: E4 (2008-08-11 21:06:30)

COCDA_0052: 2008-10-01 02:00:50 -> 2008-11-01 00:01:49
  Flybys inside COCDA_0052: E5 (2008-10-09 19:07:00), E6 (2008-10-31 17:15:30)

COCDA_0057: 2009-10-01 00:12:56 -> 2009-12-31 23:54:25
  Flybys inside COCDA_0057: E7 (2009-11-02 07:42:00)

COCDA_0059: 2010-04-01 00:35:26 -> 2010-06-30 23:35:06
  Flybys inside COCDA_0059: E9 (2010-04-28 19:09:30)

COCDA_0061: 2010-10-01 03:41:14 -> 2010-12-31 22:29:26
  Flybys inside COCDA_0061: E12 (2010-11-30 12:06:00), E13 (2010-12-21 13:11:00)

COCDA_0069: 2011-10-01 01:35:37 -> 2011-10-31 22:46:35
  Flybys inside COCDA_0069: E14 (2011-10-01 18:40:00)

COCDA_0

In [ ]:
import os
import re
import pandas as pd

# List of flybys: (name, UTC event time string)
flybys = [
    ("E3",  "2008-03-12 19:06:45"), #Confirmed YDong
    ("E4",  "2008-08-11 21:06:30"), # Confirmed
    ("E5",  "2008-10-09 19:07:00"), # Confirmed
    ("E6",  "2008-10-31 17:15:30"), #Confirmed
    ("E7",  "2009-11-02 07:42:00"), #Confirmed
    ("E9",  "2010-04-28 00:10:00"), #Half confirmed
    ("E12", "2010-11-30 11:54:00"), # Half confirmed
    ("E13", "2010-12-21 01:08:00"), #Half confirmed
    ("E14", "2011-10-01 13:52:26"), # Confirmed
    ("E17", "2012-03-27 18:30:09"), # Confirmed
    ("E18", "2012-04-14 14:01:30"), #Confirmed
    ("E19", "2012-05-02 09:31:00"), # Half confirmed
    ("E21", "2015-10-28 15:22:42"), #Half confirmed
]

# Column names from the label above (28 fields)
col_names = [
    "EVENT_CODE", "EVC", "SYC", "TP", "STAT", "OBS_TIME", "SCLK", "HD_CLK", "CLK",
    "BIG_M1", "BIG_M2", "BIG_M3", "BIG_M4", 
    "SMALL_M1", "SMALL_M2", "SMALL_M3", "SMALL_M4",
    "BIG_CM1", "BIG_CM2", "BIG_CM3", "BIG_CM4",
    "SMALL_CM1", "SMALL_CM2", "SMALL_CM3", "SMALL_CM4",
    "QUALITY_CODE", "THRESHOLD_MASS", "THRESHOLD_DIAMETER",
]

na_values = {
    "THRESHOLD_MASS": ["0.0E+00"],
    "THRESHOLD_DIAMETER": ["-99.9"],
}


base_dir = "../DataSet/cassini_high_rate_detector/data_science/raw"

def load_flyby_events(flybys, base_dir=base_dir):
    dfs = {}
    for name, utc_str in flybys:
        t0 = pd.Timestamp(utc_str)
        year = t0.year
        doy = t0.dayofyear

        subdir = os.path.abspath(os.path.join(base_dir, str(year)))

        loaded = False
        # Search all .tab files in this year’s folder
        for file in os.listdir(subdir):
            # Handle both raw (hrd_YYYY_DDD_DDD.tab) and processed (hrd_YYYY_DDD_DDD_prc.tab)
            # and also alternative formats like YYYY_hrd_DDD_DDD.tab
            match = re.search(r"(\d{3})_(\d{3})(?:_prc)?\.tab$", file)
            if match and str(year) in file:
                start_doy, end_doy = map(int, match.groups())
                if start_doy <= doy <= end_doy:
                    tab_file = os.path.join(subdir, file)
                    df = pd.read_csv(
                        tab_file, sep=r'\s+', names=col_names,
                        engine='python', comment='#', skip_blank_lines=True, na_values=na_values,
                    )
                    df["OBS_TIME"] = pd.to_datetime(df["OBS_TIME"], format="%Y-%jT%H:%M:%S.%f", errors='coerce')
                    dfs[name] = df
                    print(f"Loaded {len(df)} rows for {name} from {tab_file}")
                    loaded = True
                    break
        if not loaded:
            print(f"Flyby {name}: No suitable file found for year {year} DOY {doy}")
    return dfs

# Usage:
flyby_dfs = load_flyby_events(flybys)

Loaded 54424 rows for E3 from c:\Users\jerem\Desktop\00FinalProjectWrapper\DataSet\cassini_high_rate_detector\data_science\raw\2008\hrd_2008_070_079.tab
Flyby E4: No suitable file found for year 2008 DOY 224
Loaded 107757 rows for E5 from c:\Users\jerem\Desktop\00FinalProjectWrapper\DataSet\cassini_high_rate_detector\data_science\raw\2008\hrd_2008_280_289.tab
Loaded 327165 rows for E6 from c:\Users\jerem\Desktop\00FinalProjectWrapper\DataSet\cassini_high_rate_detector\data_science\raw\2008\hrd_2008_300_309.tab
Loaded 3208 rows for E7 from c:\Users\jerem\Desktop\00FinalProjectWrapper\DataSet\cassini_high_rate_detector\data_science\raw\2009\hrd_2009_300_309.tab
Loaded 17 rows for E9 from c:\Users\jerem\Desktop\00FinalProjectWrapper\DataSet\cassini_high_rate_detector\data_science\raw\2010\hrd_2010_110_119.tab
Loaded 3152 rows for E12 from c:\Users\jerem\Desktop\00FinalProjectWrapper\DataSet\cassini_high_rate_detector\data_science\raw\2010\hrd_2010_330_339.tab
Loaded 1779 rows for E13 from

In [7]:
# Assume flyby_dfs is the dict of single-flyby DataFrames loaded by previous code
all_events = []
flyby_window_counts = {}

for name, t_peak_str in flybys:
    if name not in flyby_dfs:
        print(f"{name}: no data loaded")
        continue

    df = flyby_dfs[name].copy()
    t_peak = pd.Timestamp(t_peak_str)
    # Calculate time difference in seconds relative to plume peak
    df["DT_PEAK_SEC"] = (df["OBS_TIME"] - t_peak).dt.total_seconds()
    window_mask = df["DT_PEAK_SEC"].abs() <= 60  # ±5 minutes (300 s)
    n_pts = window_mask.sum()
    flyby_window_counts[name] = n_pts
    print(f"{name}: {n_pts} events within ±60s of {t_peak_str}")
    # Optionally store only those points if you want to concatenate
    df_win = df[window_mask].copy()
    df_win["FLYBY"] = name
    all_events.append(df_win)

# If you want to concatenate all selected events into one DataFrame:
all_plume_events = pd.concat(all_events, ignore_index=True)

# Optionally see the counts as a summary DataFrame:
summary = pd.DataFrame(list(flyby_window_counts.items()), columns=["Flyby", "Count_±60s"])
print(summary)

E3: 0 events within ±60s of 2008-03-12 19:06:45
E4: no data loaded
E5: 0 events within ±60s of 2008-10-09 19:07:00
E6: 0 events within ±60s of 2008-10-31 17:15:30
E7: 976 events within ±60s of 2009-11-02 07:42:00
E9: 0 events within ±60s of 2010-04-28 00:10:16
E12: 0 events within ±60s of 2010-11-30 11:54:00
E13: 143 events within ±60s of 2010-12-21 01:08:00
E14: 2182 events within ±60s of 2011-10-01 13:52:26
E17: 0 events within ±60s of 2012-03-27 18:30:09
E18: 0 events within ±60s of 2012-04-14 14:01:30
E19: 0 events within ±60s of 2012-05-02 09:31:00
E21: 0 events within ±60s of 2015-10-28 15:22:42
   Flyby  Count_±60s
0     E3           0
1     E5           0
2     E6           0
3     E7         976
4     E9           0
5    E12           0
6    E13         143
7    E14        2182
8    E17           0
9    E18           0
10   E19           0
11   E21           0
